In [ ]:
import arcgis
import getpass
import pandas as pd
import pygsheets
import shutil

from arcgis.features import GeoAccessor, GeoSeriesAccessor
from pathlib import Path

In [ ]:
#: Load Google Sheet using pygsheets
client = pygsheets.authorize(service_file=r'c:\gis\git\pj-uorg\src\uorg\sheets-sa.json')
sheet_url = "https://docs.google.com/spreadsheets/d/1LbHWR0_wADXdw-10nXO8HkBC62-5rgTlscv8mzTnhmE/edit?usp=sharing"
new_data = client.open_by_url(sheet_url).worksheets()[0].get_as_df()
new_data.head()

In [ ]:
#: Compute the favorite game, fix the county names
new_data["favorite"] = new_data[["Legend of Zelda", "Ocarina of Time", "Breath of the Wild"]].idxmax(axis=1)
new_data["County"] = new_data["County"].str.replace("County", "").str.strip().str.upper()
new_data.head()

In [ ]:
#: Set up some references
username = getpass.getpass(prompt="Username: ")
gis = arcgis.gis.GIS("https://www.arcgis.com", username)
feature_layer = arcgis.features.FeatureLayer.fromitem(gis.content.get("75ca1488109d4286b16b5df00d5cc70c"))
gdb_path = Path(r"c:\temp\dev_summit.gdb")  #: GDB must already exist

In [ ]:
#: Load the existing Data
live_df = feature_layer.query(as_df=True)
live_df.head()

In [ ]:
#: Update the favorite game based on the new data
live_df["favorite_zelda"] = live_df.merge(new_data[["County", "favorite"]], left_on="name", right_on="County")["favorite"]
live_df.head()

In [ ]:
#: Save the updated data to gdb
live_df.spatial.to_featureclass(location=gdb_path / "dev_summit", overwrite=True)

In [ ]:
#: Zip the GDB
#: You may fight lockfile permissions here; if so, close/reload the interpreter and start from this point
zipped_gdb_path = shutil.make_archive(gdb_path, 'zip', root_dir=gdb_path.parent, base_dir=gdb_path.name)

In [ ]:
#: Upload the zipped GDB 
gdb_item = gis.content.add(item_properties={'type': 'File Geodatabase', 'title': 'test gdb', 'snippet': 'test gdb upload data'}, data=zipped_gdb_path)

In [ ]:
#: Use .append to update the data
result, messages = feature_layer.append(item_id=gdb_item.id, upload_format='filegdb', source_table_name='dev_summit', rollback=True, return_messages=True, upsert=True)

In [ ]:
messages

In [ ]:
#: Be sure to delete the gdb from AGOL so we can re-upload in the future
gdb_item.delete()